In [ ]:
import pandas as pd


In [ ]:

with open('ISF.L.yahoo.txt', 'r') as f:
    lines = f.readlines()

filtered_lines = [line for line in lines if 'Dividend' not in line]
df = pd.read_csv(pd.io.common.StringIO(''.join(filtered_lines)), sep='\t')

df.columns = df.columns.str.strip()
df.rename(columns={'Adj Close': 'AdjClose'}, inplace=True)

df['Date'] = df['Date'].str.replace('Sept', 'Sep')
df['Date'] = pd.to_datetime(df['Date'], format='%d %b %Y')

df = df.set_index('Date').sort_index()

df.describe()

In [ ]:
df['AdjClose'].plot()